# 🎯 GeoProj HD - Corrección de Distorsión

✅ **Resolución Original** | ✅ **Sin CUDA** | ✅ **Intensidad Ajustable**

---

## 🚀 Setup Rápido

1. GPU: Settings → GPU T4
2. Dataset: + Add Data → Modelos + Imágenes
3. Run All

⏱️ Tiempo: ~3-5 min/imagen


## 1️⃣ Instalar Dependencias


In [ ]:
%%capture --no-stderr
# Instalar versiones compatibles

!pip uninstall numpy opencv-python -y -q 2>/dev/null
!pip install numpy==1.26.4 opencv-python==4.7.0.72 -q
!pip install torch torchvision scipy matplotlib Pillow scikit-image -q


In [ ]:
# Verificar
import torch, numpy as np, cv2

print(f"✅ NumPy: {np.__version__} | OpenCV: {cv2.__version__}")
print(f"✅ GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


## 2️⃣ Clonar Código


In [ ]:
import os, shutil

for d in ['GeoProj']: 
    if os.path.exists(d): shutil.rmtree(d)

!git clone https://github.com/gallobruno81/GeoProj.git -q
os.chdir('GeoProj')
print(f"✅ Código: {os.getcwd()}")


## 3️⃣ Configurar Modelos

**Descarga**: https://drive.google.com/open?id=1Tdi92IMA-rrX2ozdUMvfiN0jCZY7wIp_


In [ ]:
os.makedirs('models', exist_ok=True)

models = ['model_en.pkl', 'model_de.pkl', 'model_class.pkl']
found = []

if os.path.exists('/kaggle/input'):
    for root, _, files in os.walk('/kaggle/input'):
        for m in models:
            if m in files:
                shutil.copy2(os.path.join(root, m), f'models/{m}')
                found.append(m)

print(f"{'✅' if len(found)==3 else '⚠️'} Modelos: {len(found)}/3")


## 4️⃣ Cargar Imágenes


In [ ]:
from IPython.display import Image as IPImage, display
from PIL import Image

images = []
if os.path.exists('/kaggle/input'):
    for root, _, files in os.walk('/kaggle/input'):
        for f in files:
            if f.lower().endswith(('.jpg', '.png', '.bmp')):
                if not os.path.exists(f):
                    shutil.copy2(os.path.join(root, f), f)
                    images.append(f)

print(f"✅ {len(images)} imagen(es)")
for img in images: display(IPImage(filename=img, width=400))


## 5️⃣ Procesar


In [ ]:
import time

if images:
    print(f"⚙️ Procesando {len(images)}...\n")
    start = time.time()
    
    for img in images:
        !python inference_hd.py --input_image "{img}" --output_dir results --model_dir models
    
    print(f"\n✅ Completado en {time.time()-start:.1f}s")


## 6️⃣ Aplicar Corrección

**FLOW_SCALE**: 1.0=normal | 1.5=fuerte ⭐ | 2.0=muy fuerte


In [ ]:
FLOW_SCALE = 1.5  # 👈 AJUSTA AQUÍ

flows = [f for f in os.listdir('results') if '_flow_hd.npy' in f]

if flows:
    print(f"🔧 Rectificando con {FLOW_SCALE}x...\n")
    start = time.time()
    
    for flow in flows:
        base = flow.replace('_flow_hd.npy', '')
        !python rectify_hd.py --img_path "results/{base}_original.jpg" --flow_path "results/{flow}" --output_dir results --flow_scale {FLOW_SCALE}
    
    print(f"\n✅ Completado en {time.time()-start:.1f}s")


## 7️⃣ Ver Resultados


In [ ]:
corrected = [f for f in os.listdir('results') if '_corrected_hd.png' in f]

for corr in corrected:
    base = corr.replace('_corrected_hd.png', '')
    
    # Comparación
    comp = f'results/{base}_comparison_simple_hd.png'
    if os.path.exists(comp):
        display(IPImage(filename=comp, width=1200))
    
    # Corregida
    display(IPImage(filename=f'results/{corr}', width=600))
    
    # Flow
    flow = f'results/{base}_flow_viz_hd.png'
    if os.path.exists(flow):
        display(IPImage(filename=flow, width=900))


## 8️⃣ Descargar


In [ ]:
!zip -r -q results_hd.zip results/

if os.path.exists('results_hd.zip'):
    size = os.path.getsize('results_hd.zip') / (1024*1024)
    print(f"✅ results_hd.zip ({size:.2f} MB)")
    print("\n📥 Save Version → Output → Descarga")


---

## 🎉 ¡Listo!

**Tips**: Ajusta `FLOW_SCALE` en celda 6 | Más imágenes: desde celda 4

**GitHub**: https://github.com/gallobruno81/GeoProj

*GeoProj HD v2 - Octubre 2025*
